# 5. Live Camera Detection (ONNX) - Bicep Curl

**Objective:**
1. Load the ONNX models (left & right arm) and JSON scalers.
2. Initialize MediaPipe Pose.
3. Capture video from webcam.
4. For each frame:
    a. Perform pose estimation.
    b. Extract arm landmarks.
    c. Calculate angles using `GeometryUtils`.
    d. Manually scale features using JSON scaler parameters.
    e. Perform inference using the ONNX model to classify form.
    f. Implement rep counting and feedback logic.
    g. Display results on the frame.

**Note:** This notebook will run an OpenCV window for live camera feed. Ensure you have a webcam connected. To stop the feed, press 'q' in the OpenCV window.

In [17]:
import cv2
import mediapipe as mp
import numpy as np
import json
from collections import deque
import time
import onnxruntime
import os
import sys

# Add utils directory to sys.path
module_path = os.path.abspath(os.path.join('../../..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.geometry_utils import GeometryUtils # For calculate_angle

# IPython display for showing images in notebook (optional, mainly for static checks)
from IPython.display import display, Image
import threading # For running OpenCV loop in a way that doesn't block notebook kernel

## 5.1 Constants and Configuration

In [18]:
# --- Paths ---
ONNX_MODEL_DIR = "../models/onnx/"

# --- Model Configuration ---
# ==> example: "NB" for Normalized Body, "RF" for Random Forest, etc.
MODEL_TYPE_LEFT = "NB"
MODEL_TYPE_RIGHT = "NB"

# --- MediaPipe Configuration ---
MP_MIN_DETECTION_CONFIDENCE = 0.6
MP_MIN_TRACKING_CONFIDENCE = 0.6
MP_MODEL_COMPLEXITY = 1 # 0, 1, or 2. Higher is more accurate but slower.

# --- Frame Configuration ---
FRAME_WIDTH = 960 # If too large, might be slow
FRAME_HEIGHT = 540

# --- Arm Processing Logic Configuration ---
MIN_VISIBILITY = 0.6 # Stricter than original script's 0.5 for potentially better quality angles
SMOOTH_WINDOW_SIZE = 5 # For angle smoothing

# Rep Counting Thresholds & Parameters (same as your provided detection script)
ELBOW_ANGLE_UP_THRESHOLD = 85     # Angle to consider 'UP' state (e.g., elbow < 85 degrees)
ELBOW_ANGLE_DOWN_THRESHOLD = 140  # Angle to consider 'DOWN' state (e.g., elbow > 140 degrees)
SHOULDER_TOLERANCE = 25           # Max allowed deviation from neutral shoulder angle
SHOULDER_CALIBRATION_ANGLE_MIN = 140 # Min elbow angle to start calibrating shoulder
REP_COOLDOWN_SECONDS = 0.5        # Min time between reps
MIN_REP_DURATION_SECONDS = 0.4    # Min duration for a valid rep
MAX_REP_DURATION_SECONDS = 3.0    # Max duration for a valid rep
ELBOW_EXTENSION_TARGET = 140      # For feedback: target elbow angle in down phase
ELBOW_SQUEEZE_TARGET = 75         # For feedback: target elbow angle in up phase

# --- Display Configuration ---
FONT = cv2.FONT_HERSHEY_SIMPLEX
COLOR_CORRECT_FORM = (0, 255, 0)      # Green
COLOR_INCORRECT_FORM = (0, 0, 255)    # Red
COLOR_NEUTRAL = (255, 255, 255)     # White
COLOR_FEEDBACK = (255, 200, 0)      # Light Blue/Orange for feedback text
COLOR_VIS_LOW = (0, 165, 255)       # Orange for low visibility
COLOR_MODEL_ERROR = (128, 0, 128)   # Purple for model/scaler load errors

## 5.2 Initialize MediaPipe and GeometryUtils

In [19]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose_detector = mp_pose.Pose(
    min_detection_confidence=MP_MIN_DETECTION_CONFIDENCE,
    min_tracking_confidence=MP_MIN_TRACKING_CONFIDENCE,
    model_complexity=MP_MODEL_COMPLEXITY
)

geo_utils = GeometryUtils()

## 5.3 Load ONNX Models and JSON Scalers

In [20]:
def load_onnx_model_and_json_scaler(side, model_dir, model_type):
    onnx_model_path = os.path.join(model_dir, f"{model_type}_model_{side}.onnx")
    scaler_json_path = os.path.join(model_dir, f"scaler_{side}.json")
    
    ort_session, scaler_params, input_name = None, None, None

    # Load ONNX model
    if os.path.exists(onnx_model_path):
        try:
            ort_session = onnxruntime.InferenceSession(onnx_model_path)
            input_name = ort_session.get_inputs()[0].name
            print(f"ONNX model for {side} loaded. Input name: '{input_name}'")
        except Exception as e:
            print(f"Error loading ONNX model for {side} from {onnx_model_path}: {e}")
    else:
        print(f"ONNX model file not found for {side}: {onnx_model_path}")

    # Load scaler parameters from JSON
    if os.path.exists(scaler_json_path):
        try:
            with open(scaler_json_path, "r") as f_scaler_json:
                scaler_data = json.load(f_scaler_json)
            # Ensure mean and scale are numpy arrays and correctly shaped for broadcasting
            scaler_params = {
                'mean': np.array(scaler_data["mean"], dtype=np.float32).reshape(1, -1),
                'scale': np.array(scaler_data["scale"], dtype=np.float32).reshape(1, -1)
            }
            # Verify scale is not zero
            if np.any(scaler_params['scale'] == 0):
                print(f"Warning: Scaler for {side} has zero values in 'scale'. Replacing with 1.0 to avoid division by zero.")
                scaler_params['scale'][scaler_params['scale'] == 0] = 1.0
            print(f"JSON scaler parameters for {side} loaded.")
            # print(f"  Scaler mean: {scaler_params['mean']}, Scaler scale: {scaler_params['scale']}")
        except Exception as e:
            print(f"Error loading/parsing scaler JSON for {side} from {scaler_json_path}: {e}")
    else:
        print(f"Scaler JSON file not found for {side}: {scaler_json_path}")
        
    return ort_session, scaler_params, input_name

# Load for both arms
ort_session_left, scaler_params_left, input_name_left = load_onnx_model_and_json_scaler("left", ONNX_MODEL_DIR, MODEL_TYPE_LEFT)
ort_session_right, scaler_params_right, input_name_right = load_onnx_model_and_json_scaler("right", ONNX_MODEL_DIR, MODEL_TYPE_RIGHT)

ONNX model for left loaded. Input name: 'float_input'
JSON scaler parameters for left loaded.
ONNX model for right loaded. Input name: 'float_input'
JSON scaler parameters for right loaded.


## 5.4 Arm Processor Class

In [21]:
class ArmProcessor:
    def __init__(self, side, ort_session, scaler_params, input_name, geo_utils):
        self.side = side
        self.ort_session = ort_session
        self.scaler_mean = scaler_params['mean'] if scaler_params else None
        self.scaler_scale = scaler_params['scale'] if scaler_params else None
        self.input_name = input_name
        self.geo_utils = geo_utils

        self.is_ready = (self.ort_session is not None and
                         self.scaler_mean is not None and
                         self.scaler_scale is not None and
                         self.input_name is not None)

        self.state = "DOWN"  # Initial state: 'DOWN' or 'UP'
        self.reps = 0
        self.angle_history = deque(maxlen=SMOOTH_WINDOW_SIZE)
        self.shoulder_neutral = None # Calibrated neutral shoulder angle
        self.shoulder_angles_for_calibration = deque(maxlen=30) # Buffer for calibration
        self.last_rep_time = 0
        self.time_entered_up_state = 0
        self.rep_quality_feedback = "" # e.g., "Good rep!", "Too fast!"
        self.last_form_feedback_model = "" # Feedback from the ONNX model

        if not self.is_ready:
            print(f"ArmProcessor for {self.side} side is NOT ready. ONNX Model, scaler params, or input name missing.")

    def calibrate_shoulder(self, current_shoulder_angle):
        self.shoulder_angles_for_calibration.append(current_shoulder_angle)
        # Calibrate once enough samples are collected and stable
        if len(self.shoulder_angles_for_calibration) >= 15: # Require at least 15 samples
            # Use median to be robust to outliers
            self.shoulder_neutral = np.median(list(self.shoulder_angles_for_calibration))
            # print(f"[{self.side}] Shoulder calibrated to: {self.shoulder_neutral:.1f}°")
            self.shoulder_angles_for_calibration.clear() # Clear after calibration or if it drifts too much

    def get_form_feedback_rules(self, elbow_angle_smoothed, shoulder_angle_smoothed):
        feedback_messages = []
        # Shoulder stability feedback
        if self.shoulder_neutral is not None:
            shoulder_deviation = shoulder_angle_smoothed - self.shoulder_neutral
            if abs(shoulder_deviation) > SHOULDER_TOLERANCE:
                feedback_messages.append("Stabilize shoulder" if shoulder_deviation < 0 else "Avoid leaning back")
        else:
            feedback_messages.append("Calibrating shoulder...")

        # Elbow extension/squeeze feedback based on state
        if self.state == "DOWN" and elbow_angle_smoothed < (ELBOW_EXTENSION_TARGET - 15):
            feedback_messages.append("Extend arm fully")
        elif self.state == "UP" and elbow_angle_smoothed > (ELBOW_SQUEEZE_TARGET + 15):
            feedback_messages.append("Squeeze bicep more")

        if not feedback_messages:
            return "Good form!", COLOR_CORRECT_FORM
        else:
            return " | ".join(feedback_messages), COLOR_FEEDBACK

    def process_landmarks(self, landmarks_mp, frame_shape_wh):
        frame_width, frame_height = frame_shape_wh
        self.rep_quality_feedback = "" # Reset rep quality feedback each frame

        if not self.is_ready:
            return None, f"{self.side.capitalize()} Model/Scaler Not Loaded", COLOR_MODEL_ERROR, (0,0)

        # Define MediaPipe landmarks for the current arm
        lm_indices = {
            "shoulder": mp_pose.PoseLandmark[f"{self.side.upper()}_SHOULDER"],
            "elbow": mp_pose.PoseLandmark[f"{self.side.upper()}_ELBOW"],
            "wrist": mp_pose.PoseLandmark[f"{self.side.upper()}_WRIST"],
            "hip": mp_pose.PoseLandmark[f"{self.side.upper()}_HIP"]
        }

        try:
            # Get landmark coordinates and visibility
            actual_lms = {name: landmarks_mp[idx.value] for name, idx in lm_indices.items()}
            
            # Check visibility of all key landmarks
            if not all(lm.visibility > MIN_VISIBILITY for lm in actual_lms.values()):
                # print(f"Low visibility for {self.side} arm: {[ (name, lm.visibility) for name, lm in actual_lms.items() if lm.visibility <= MIN_VISIBILITY]}")
                return None, f"{self.side.capitalize()} arm not fully visible", COLOR_VIS_LOW, (int(actual_lms['elbow'].x * frame_width) if actual_lms['elbow'].visibility > 0.1 else 0 , int(actual_lms['elbow'].y * frame_height) if actual_lms['elbow'].visibility > 0.1 else 0)

            # Extract [x, y] coordinates
            coords = {name: [lm.x, lm.y] for name, lm in actual_lms.items()}

            # Calculate current angles
            current_elbow_angle = self.geo_utils.calculate_angle(coords["shoulder"], coords["elbow"], coords["wrist"])
            current_shoulder_angle = self.geo_utils.calculate_angle(coords["elbow"], coords["shoulder"], coords["hip"])
            
            if np.isnan(current_elbow_angle) or np.isnan(current_shoulder_angle):
                return None, f"Angle calc error {self.side}", COLOR_INCORRECT_FORM, (int(coords['elbow'][0] * frame_width), int(coords['elbow'][1] * frame_height))

            # Smooth angles
            self.angle_history.append((current_elbow_angle, current_shoulder_angle))
            if len(self.angle_history) < SMOOTH_WINDOW_SIZE:
                elbow_angle_smoothed, shoulder_angle_smoothed = current_elbow_angle, current_shoulder_angle
            else:
                elbow_angle_smoothed, shoulder_angle_smoothed = np.mean(self.angle_history, axis=0)

            # Shoulder calibration logic
            # Calibrate only when arm is relatively straight (down phase) to get a neutral posture
            if self.state == "DOWN" and elbow_angle_smoothed > SHOULDER_CALIBRATION_ANGLE_MIN and (self.shoulder_neutral is None or len(self.shoulder_angles_for_calibration)>0):
                self.calibrate_shoulder(shoulder_angle_smoothed)
            elif self.shoulder_neutral is not None and abs(shoulder_angle_smoothed - self.shoulder_neutral) > SHOULDER_TOLERANCE * 2: # If shoulder deviates too much, reset calibration buffer
                self.shoulder_angles_for_calibration.clear() # Encourage re-calibration if posture changes significantly

            # --- ONNX Model Prediction ---
            features_unscaled = np.array([[elbow_angle_smoothed, shoulder_angle_smoothed]], dtype=np.float32)
            features_scaled = (features_unscaled - self.scaler_mean) / self.scaler_scale
            
            input_feed = {self.input_name: features_scaled}
            pred_onnx = self.ort_session.run(None, input_feed)
            # Assuming model output is [prediction_class_index], and 1=Correct, 0=Incorrect
            model_form_prediction = pred_onnx[0][0] # This is the predicted class (0 or 1)
            
            is_form_correct_model = (model_form_prediction == 1) # 1 for 'C' (Correct)
            self.last_form_feedback_model = "Form OK (Model)" if is_form_correct_model else "Fix Form (Model)"
            model_feedback_color = COLOR_CORRECT_FORM if is_form_correct_model else COLOR_INCORRECT_FORM

            # --- Rep Counting & Rule-Based Feedback ---
            current_time = time.time()
            rule_based_feedback_text, rule_based_feedback_color = self.get_form_feedback_rules(elbow_angle_smoothed, shoulder_angle_smoothed)
            
            # Combine model feedback and rule-based feedback
            # Priority: If model says form is incorrect, that takes precedence for display color.
            final_feedback_text = f"{self.last_form_feedback_model} | {rule_based_feedback_text}"
            final_feedback_color = model_feedback_color if not is_form_correct_model else rule_based_feedback_color

            # Rep counting logic (only if model predicts good form or if we allow reps with minor rule infractions)
            # For simplicity, let's allow rep counting even if rule-based feedback is active, but model must say OK.
            if is_form_correct_model and (current_time - self.last_rep_time > REP_COOLDOWN_SECONDS):
                if self.state == "DOWN" and elbow_angle_smoothed < ELBOW_ANGLE_UP_THRESHOLD:
                    self.state = "UP"
                    self.time_entered_up_state = current_time
                elif self.state == "UP" and elbow_angle_smoothed > ELBOW_ANGLE_DOWN_THRESHOLD:
                    self.state = "DOWN"
                    self.last_rep_time = current_time
                    if self.time_entered_up_state != 0:
                        rep_duration = current_time - self.time_entered_up_state
                        if MIN_REP_DURATION_SECONDS <= rep_duration <= MAX_REP_DURATION_SECONDS:
                            self.reps += 1
                            self.rep_quality_feedback = "Good rep!"
                        elif rep_duration < MIN_REP_DURATION_SECONDS:
                            self.rep_quality_feedback = "Rep too fast!"
                        else:
                            self.rep_quality_feedback = "Rep too slow!"
                    else: # Should not happen if logic is correct
                        self.rep_quality_feedback = "Timing error"
                    self.time_entered_up_state = 0 # Reset for next rep
            elif not is_form_correct_model and self.state == "UP":
                # If form breaks mid-rep (UP state), reset state to DOWN to prevent counting bad rep completion
                self.state = "DOWN"
                self.time_entered_up_state = 0
                self.rep_quality_feedback = "Form broke mid-rep!"

            # Position for angle text (near elbow)
            angle_text_coords = (int(coords['elbow'][0] * frame_width) + 10, int(coords['elbow'][1] * frame_height))
            
            return int(elbow_angle_smoothed), final_feedback_text, final_feedback_color, angle_text_coords

        except IndexError:
            # print(f"IndexError: Landmark not found for {self.side} arm.") # Expected if person not fully in frame
            return None, f"{self.side.capitalize()} landmarks missing", COLOR_VIS_LOW, (0,0)
        except Exception as e:
            import traceback
            print(f"Error processing {self.side} arm: {e}")
            # traceback.print_exc() # Uncomment for detailed stack trace
            return None, f"Error {self.side}", COLOR_INCORRECT_FORM, (0,0)

    def reset_reps(self):
        self.reps = 0
        self.state = "DOWN"
        self.angle_history.clear()
        self.shoulder_angles_for_calibration.clear()
        self.shoulder_neutral = None
        self.last_rep_time = 0
        self.time_entered_up_state = 0
        self.rep_quality_feedback = ""
        print(f"[{self.side.upper()}] Reps, state, and calibration reset.")

## 5.5 Initialize Arm Processors

In [22]:
left_processor = ArmProcessor("left", ort_session_left, scaler_params_left, input_name_left, geo_utils)
right_processor = ArmProcessor("right", ort_session_right, scaler_params_right, input_name_right, geo_utils)

## 5.6 Main Video Processing Loop

In [ ]:
stop_camera_loop = False

def video_processing_loop():
    global stop_camera_loop
    stop_camera_loop = False
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

    print("Starting Bicep Curl Trainer. Press 'q' in the OpenCV window to quit, 'r' to reset reps.")
    print(f"Target rep duration: {MIN_REP_DURATION_SECONDS:.1f}s - {MAX_REP_DURATION_SECONDS:.1f}s")
    
    cv_window_name = 'Bicep Curl AI Trainer (ONNX)'
    cv2.namedWindow(cv_window_name, cv2.WINDOW_AUTOSIZE)

    while cap.isOpened() and not stop_camera_loop:
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame or end of video.")
            if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT) and cap.get(cv2.CAP_PROP_FRAME_COUNT) > 0:
                 break
            continue

        frame_to_process = frame 

        image_rgb = cv2.cvtColor(frame_to_process, cv2.COLOR_BGR2RGB)
        image_rgb.flags.writeable = False 
        results = pose_detector.process(image_rgb)
        image_rgb.flags.writeable = True 
        output_image = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)

        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                output_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=1)
            )

        display_frame = cv2.flip(output_image, 1)

        if results.pose_landmarks:
            processors = [left_processor, right_processor]
            for i, processor in enumerate(processors):
                feedback_base_y = 70
                text_x_pos_orig = 10 if processor.side == "left" else FRAME_WIDTH - 450 
                
                angle_val, form_fb_str, form_fb_col, angle_text_coords_orig = processor.process_landmarks(
                    results.pose_landmarks.landmark,
                    (FRAME_WIDTH, FRAME_HEIGHT)
                )
                
                current_y_offset = feedback_base_y + (i * 100) 
                if processor.side == "right": current_y_offset = feedback_base_y

                if angle_val is not None and angle_text_coords_orig != (0,0):
                    text_angle = f"{angle_val}°"
                    (text_w, _), _ = cv2.getTextSize(text_angle, FONT, 0.7, 2)
                    orig_x_angle, orig_y_angle = angle_text_coords_orig
                    new_x_angle = FRAME_WIDTH - orig_x_angle - text_w
                    cv2.putText(display_frame, text_angle, (new_x_angle, orig_y_angle),
                                FONT, 0.7, form_fb_col, 2, cv2.LINE_AA)

                if form_fb_str:
                    words = form_fb_str.split(' ')
                    lines = []
                    current_line = ""
                    max_line_width_px = 420 
                    for word_idx, word in enumerate(words):
                        test_line = f"{current_line} {word}".strip()
                        (line_w, _), _ = cv2.getTextSize(test_line, FONT, 0.6, 1)
                        if line_w > max_line_width_px and current_line:
                            lines.append(current_line)
                            current_line = word
                        else:
                            current_line = test_line
                        if word_idx == len(words) - 1:
                            lines.append(current_line)
                    
                    for line_idx, line_text in enumerate(lines):
                        (text_w, _), _ = cv2.getTextSize(line_text, FONT, 0.6, 1)
                        new_x_line = FRAME_WIDTH - text_x_pos_orig - text_w
                        cv2.putText(display_frame, line_text, (new_x_line, current_y_offset + line_idx * 20),
                                   FONT, 0.6, form_fb_col, 2, cv2.LINE_AA)
                    current_y_offset += len(lines) * 20 + 5

                if processor.rep_quality_feedback:
                    rep_fb_color = COLOR_CORRECT_FORM if "Good" in processor.rep_quality_feedback else COLOR_INCORRECT_FORM
                    text_rep_quality = processor.rep_quality_feedback
                    (text_w, _), _ = cv2.getTextSize(text_rep_quality, FONT, 0.7, 2)
                    new_x_rep_quality = FRAME_WIDTH - text_x_pos_orig - text_w
                    cv2.putText(display_frame, text_rep_quality,
                                (new_x_rep_quality, current_y_offset),
                                FONT, 0.7, rep_fb_color, 2, cv2.LINE_AA)
        else:
            text_no_person = "No person detected"
            (text_w, _), _ = cv2.getTextSize(text_no_person, FONT, 1, 2)
            orig_x_no_person = FRAME_WIDTH // 2 - 150
            new_x_no_person = FRAME_WIDTH - orig_x_no_person - text_w
            cv2.putText(display_frame, text_no_person, (new_x_no_person, FRAME_HEIGHT // 2),
                        FONT, 1, COLOR_INCORRECT_FORM, 2, cv2.LINE_AA)

        text_left_reps = f"Left Reps: {left_processor.reps}"
        (text_w_left, _), _ = cv2.getTextSize(text_left_reps, FONT, 1, 2)
        orig_x_left_reps = 10
        new_x_left_reps = FRAME_WIDTH - orig_x_left_reps - text_w_left
        cv2.putText(display_frame, text_left_reps, (new_x_left_reps, 40),
                    FONT, 1, COLOR_NEUTRAL, 2, cv2.LINE_AA)
        
        text_right_reps = f"Right Reps: {right_processor.reps}"
        (text_w_right, _), _ = cv2.getTextSize(text_right_reps, FONT, 1, 2)
        orig_x_right_reps = FRAME_WIDTH - 280
        new_x_right_reps = FRAME_WIDTH - orig_x_right_reps - text_w_right
        cv2.putText(display_frame, text_right_reps, (new_x_right_reps, 40),
                    FONT, 1, COLOR_NEUTRAL, 2, cv2.LINE_AA)
        
        cv2.imshow(cv_window_name, display_frame)

        key = cv2.waitKey(5) & 0xFF
        if key == ord('q'):
            print("Quit key pressed. Stopping detection.")
            break
        elif key == ord('r'):
            print("Resetting reps for both arms.")
            left_processor.reset_reps()
            right_processor.reset_reps()
            
    cap.release()
    cv2.destroyAllWindows()

    for i in range(5):
        cv2.waitKey(1)
    print("Video processing loop finished.")

### How to Run the Live Detection:

1.  Make sure you have executed all previous cells to load models, scalers, and define classes/functions.
2.  Uncomment the line `video_processing_loop()` in the cell above (or the threading lines if you prefer non-blocking behavior, though direct call is simpler for typical notebook use).
3.  Run the cell. An OpenCV window should appear showing your webcam feed with detections.
4.  **Press 'q' in the OpenCV window** to close it and stop the detection loop.
5.  **Press 'r' in the OpenCV window** to reset rep counts and arm states.

If the OpenCV window doesn't close properly or the kernel seems stuck after pressing 'q', you might need to:
- Interrupt the kernel (from the Kernel menu in Jupyter).
- Restart the kernel.

**Stopping the loop from another cell (if using threading and `stop_camera_loop` global):**
```python
# global stop_camera_loop
# stop_camera_loop = True
# if 'video_thread' in globals() and video_thread.is_alive():
#     video_thread.join(timeout=2) # Wait for thread to finish
#     print("Video thread stopped.")
# cv2.destroyAllWindows()
```
For simplicity in a notebook, directly calling `video_processing_loop()` is often easier to manage.

In [ ]:
if ort_session_left and ort_session_right and scaler_params_left and scaler_params_right:
    print("Models and scalers seem to be loaded. Ready to start video processing.")
    video_processing_loop()
else:
    print("ONNX models or scalers are not loaded properly. Cannot start live detection.")
    print(f"  Left Session: {'OK' if ort_session_left else 'FAIL'}, Left Scaler: {'OK' if scaler_params_left else 'FAIL'}")
    print(f"  Right Session: {'OK' if ort_session_right else 'FAIL'}, Right Scaler: {'OK' if scaler_params_right else 'FAIL'}")

Models and scalers seem to be loaded. Ready to start video processing.


Starting Bicep Curl Trainer. Press 'q' in the OpenCV window to quit, 'r' to reset reps.
Target rep duration: 0.4s - 3.0s
Ignoring empty camera frame or end of video.
Video processing loop finished.
Ignoring empty camera frame or end of video.
Video processing loop finished.


After running the live detection, if the OpenCV window doesn't close cleanly or you need to force stop, you might need to interrupt or restart the Jupyter kernel.